This is the notebook I built based on the Get Started version with additional modifications **[Tabular Playground Series - Jan 2021](https://admin.kaggle.com/c/tabular-playground-series-jan-2021/overview)** competition. 

# Make the most of this notebook!


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, StackingRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

from xgboost import XGBRegressor

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
        
input_path = Path('/kaggle/input/tabular-playground-series-jan-2021/')

# Read in the data files

In [ ]:
# training data
train = pd.read_csv(input_path / 'train.csv', index_col='id')

# test data
test = pd.read_csv(input_path / 'test.csv', index_col='id')

submission = pd.read_csv(input_path / 'sample_submission.csv', index_col='id')

## Pull out the target, and make a validation split

In [ ]:
target = train.pop('target')
X_train, X_test, y_train, y_test = train_test_split(train, target, train_size=0.8)

## Ploting the dataset

Checking the correlations

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(train.corr(), annot=True, cmap="YlGnBu");

Plot of selected features of the train dataset

In [ ]:
X_train.columns

In [ ]:
sns.jointplot(X_train.cont7[:1000], y_train[:1000], alpha=0.3);

In [ ]:
sns.distplot(X_train.cont1);

In [ ]:
sns.distplot(X_train.cont14);

In [ ]:
sns.distplot(y_train);

# Models

### Gradient regressor

In [ ]:
parameters = {
    'n_estimators': 350,
    'tree_method': 'hist',
    'learning_rate': 0.03,
    'colsample_bytree': 0.9,
    'subsample': 0.9,
    'min_child_weight': 9,
    'max_depth': 11,
    'n_jobs': -1
}

In [ ]:
parameters2 = {
    'n_estimators': 350,
    'tree_method': 'exact',
    'learning_rate': 0.03,
    'colsample_bytree': 0.9,
    'subsample': 0.9,
    'min_child_weight': 9,
    'max_depth': 11,
    'n_jobs': -1
}

In [ ]:
'''model = XGBRegressor(**parameters)
model.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_test, y_test)], verbose=False)
prediction = model.predict(X_test)
mse = mean_squared_error(y_test, prediction,squared=False)
print(mse)'''

# mse = 0.700

## Using Gridsearch

In [ ]:
# from sklearn.model_selection import GridSearchCV

In [ ]:
'''param_grid = [
    {
        'n_estimators': [350, 1000],
        'learning_rate': [0.01, 0.03],
        'colsample_bytree': [0.5, 0.9]
    },
    {
        'n_estimators': [350, 1000],
        'subsample': [0.7, 0.9]
    }
]

new_model = XGBRegressor(tree_method='hist', min_child_weight=9, max_depth=11, n_jobs=-1)

grid_search = GridSearchCV(new_model, param_grid, cv=3, scoring='neg_mean_squared_error')

grid_search.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_test, y_test)], verbose=1)'''

In [ ]:
#grid_search.best_params_

In [ ]:
#grid_search.best_estimator_

In [ ]:
'''cvres = grid_search.cv_results_

for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)'''

In [ ]:
'''XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=11,
             min_child_weight=9, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=-1, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='hist', validate_parameters=1, verbosity=None)'''

Building the final model

In [ ]:
final_model = XGBRegressor(tree_method='hist', min_child_weight=9, max_depth=11, n_jobs=-1, colsample_bytree=0.5, learning_rate=0.01, n_estimators=1500)
final_model.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_test, y_test)], verbose=False)
prediction = final_model.predict(X_test)
mse = mean_squared_error(y_test, prediction,squared=False)
print(mse)

# mse=0.697679

## Ensemble methods

Comparison of the target of the test dataset with the model prediction

In [ ]:
sns.distplot(y_test)
sns.distplot(prediction);

## Preparing file for submission

In [ ]:
submission['target'] = final_model.predict(test)
submission.to_csv('xgb_reg.csv')